<a href="https://colab.research.google.com/github/wasan443/Python-for-number-theory/blob/master/Simulated_annealingOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
df_cottages = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/El Orteca Resorts .xlsx', sheet_name='Cottages')
df_reservations = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/El Orteca Resorts .xlsx', sheet_name='Reservations')

In [12]:
print(df_cottages.columns)
print(df_reservations.columns)

Index(['ID', 'Max # Pers', 'Class', 'Face South', 'Near Playground',
       'Close to the Centre', 'Near Lake ', 'Near car park',
       'Accessible for Wheelchair', 'Child Friendly', 'Dish Washer ',
       'Wi-Fi Coverage ', 'Covered Terrace'],
      dtype='object')
Index(['ID', 'Arrival Date', 'Length of Stay', '# Persons', 'Class',
       'Face South', 'Near Playground', 'Close to the Centre', 'Near Lake ',
       'Near car park', 'Accessible for Wheelchair', 'Child Friendly',
       'Dish Washer ', 'Wi-Fi Coverage ', 'Covered Terrace',
       'Cottage (Fixed)'],
      dtype='object')


In [13]:
from collections import defaultdict
from datetime import datetime, timedelta

def calculate_cost(state, pd_reservations, pd_cottages):
    num_gaps = 0
    num_friday_to_thursday_gaps = 0
    legionella_risk_periods = 0
    num_upgrades = 0

    # Group reservations per cottage
    reservations_by_cottage = defaultdict(list)
    for reservation_id, cottage_id in enumerate(state):
        reservations_by_cottage[cottage_id].append(reservation_id)

    for cottage_id, reservation_ids in reservations_by_cottage.items():
        cottage = pd_cottages.loc[cottage_id] #  cottage object bevat alle kernmerken van een cottage
        # Haal de reserveringen voor deze cottage op en sorteer ze op start date
         reservations = sorted((pd_reservations.loc[reservation_id] for reservation_id in reservation_ids),
                              key=lambda r: r['Arrival Date'])
        
        # Berken gaps
        gaps = [(reservations[i+1]['Arrival Date'] - (reservations[i]['Arrival Date'] + timedelta(days=int(reservations[i]['Length of Stay']))))
                for i in range(len(reservations)-1)] # om niet de index uit of range
        num_gaps += len(gaps)
        
        # Bereken het aantal Fridag tot donderdag gaps
        #num_friday_to_thursday_gaps += len([gap for gap in gaps if gap.days >= 7 and reservations[i]['Arrival Date'].weekday() == 4 and (reservations[i]['Arrival Date'] + timedelta(days=reservations[i]['Length of Stay'])).weekday() == 3 for i in range(len(reservations)-1)])
        num_friday_to_thursday_gaps += len([gap for i, gap in enumerate(gaps) if gap.days >= 7 and reservations[i]['Arrival Date'].weekday() == 4 and (reservations[i]['Arrival Date'] + timedelta(days=int(reservations[i]['Length of Stay']))).weekday() == 3 for i in range(len(reservations)-1)])

        
        # Bereken legionella periods
        legionella_risk_periods += len([gap for gap in gaps if gap.days > 21])

        # Bereken upgrades
        for reservation_id in reservation_ids:
            reservation = pd_reservations.loc[reservation_id]
            if cottage['Class'] > reservation['Class']:# cottage['Max # Pers'] > reservation['# Persons'] dit is geen upgrade
                num_upgrades += 1

    return 6 * num_gaps - 3 * num_friday_to_thursday_gaps + 12 * legionella_risk_periods + num_upgrades


In [21]:
def initial_assignment_random(pd_reservations, pd_cottages):
    state = []
    for _, reservation in pd_reservations.iterrows():
        # # Vind cottages die de reservering kunnen accommoderen
        suitable_cottages = pd_cottages[(pd_cottages['Max # Pers'] >= reservation['# Persons']) & 
                                         (pd_cottages['Class'] >= reservation['Class'])]

        # Als er geen geschikte cottage is gevonden, wijs dan toe aan een willekeurige cottage
        if suitable_cottages.empty:
            suitable_cottage = random.choice(pd_cottages.index.to_list())
        else:
            suitable_cottage = random.choice(suitable_cottages.index.to_list())
        
        state.append(suitable_cottage)

    return state


In [15]:
def is_valid_swap(state, res1, res2, pd_cottages, pd_reservations):
    # Haal de cottages op voor de twee reserveringen
    cottage1 = state[res1]
    cottage2 = state[res2]

    # Haal de reserveringsdetails op
    reservation1 = pd_reservations.loc[res1]
    reservation2 = pd_reservations.loc[res2]
    #print(f"reservation 1 details: {reservation1}")

    # Haal de cottagesdetails op
    cottage_details1 = pd_cottages.loc[cottage1]
    cottage_details2 = pd_cottages.loc[cottage2]
    #print(f"cottage_details1: {cottage_details1}")
    
    
    # Controleer of cottage1 reservering2 kan accommoderen
    if (cottage_details1['Max # Pers'] < reservation2['# Persons'] or
        cottage_details1['Class'] < reservation2['Class']):
        return False

    # Controleer of cottage2 reservering1 kan accommoderen
    if (cottage_details2['Max # Pers'] < reservation1['# Persons'] or
        cottage_details2['Class'] < reservation1['Class']):
        return False

    return True


In [16]:
def get_neighbor(state, pd_reservations, pd_cottages):
    new_state = state.copy()

    # Wij blijven proberen tot we een geldige swap vinden
    while True:
        # Selecteer twee willekeurige reserveringen
        res1, res2 = random.sample(pd_reservations.index.to_list(), 2)
        
        # Controleer of de swap geldig is volgens de capaciteiten van de cottages en klassen
        if is_valid_swap(new_state, res1, res2, pd_cottages, pd_reservations):
            new_state[res1], new_state[res2] = new_state[res2], new_state[res1]
            break

    return new_state



In [17]:
import math
import random

def simulated_annealing(pd_reservations, pd_cottages, initial_temperature, cooling_rate, max_iterations):
    # Initialiseer de huidige state en bereken de kosten 
    current_state = initial_assignment_random(pd_reservations, pd_cottages)
    current_cost = calculate_cost(current_state, pd_reservations, pd_cottages)

    # maak de initiële state als de beste state
    best_state = current_state
    best_cost = current_cost

    # 
    temperature = initial_temperature

    # Loop for the specified number of iterations
    for iteration in range(max_iterations):
        # Get a new neighbor state and calculate its cost
        neighbor_state = get_neighbor(current_state, pd_reservations, pd_cottages)# na swap carculate cost again
        neighbor_cost = calculate_cost(neighbor_state, pd_reservations, pd_cottages) 

        # If the neighbor state is better, accept it
        if neighbor_cost < current_cost:
            current_state = neighbor_state
            current_cost = neighbor_cost

            # If this state is better than the best state, update the best state
            if current_cost < best_cost:
                best_state = current_state
                best_cost = current_cost

        # If the neighbor state is worse, maybe accept it
        else:
            if random.uniform(0, 1) < math.exp((current_cost - neighbor_cost) / temperature):
                current_state = neighbor_state
                current_cost = neighbor_cost

        # Cool down
        temperature *= cooling_rate

    return best_state, best_cost


In [ ]:
import math
import random

def simulated_annealing(pd_reservations, pd_cottages, initial_temperature, cooling_rate, max_iterations):
    # Initialiseer de huidige state en bereken de kosten 
    current_state = initial_assignment_random(pd_reservations, pd_cottages)
    current_cost = calculate_cost(current_state, pd_reservations, pd_cottages)

    # maak de initiële state als de beste state
    best_state = current_state
    best_cost = current_cost

    # Initialiseer de temperatuur
    temperature = initial_temperature

    # Loop voor het gespecificeerde aantal iteraties
    for iteration in range(max_iterations):
         # Haal een nieuwe neigbor state op en bereken de kosten 
        neighbor_state = get_neighbor(current_state, pd_reservations, pd_cottages)# na swap carculate cost again
        neighbor_cost = calculate_cost(neighbor_state, pd_reservations, pd_cottages) 

        # Als de neighbor state beter is, accepteer het
        if neighbor_cost < current_cost:
            current_state = neighbor_state
            current_cost = neighbor_cost

            # Als deze state beter is dan de beste state, update de beste state
            if current_cost < best_cost:
                best_state = current_state
                best_cost = current_cost

        # Als de neighbor state slechter is, accepteer het 
        else:
            if random.uniform(0, 1) < math.exp((current_cost - neighbor_cost) / temperature):
                current_state = neighbor_state
                current_cost = neighbor_cost

        # afkoelen
        temperature *= cooling_rate

    return best_state, best_cost


In [18]:
# Definieer parameters voor de gesimuleerde annealing functie
initial_temperature = 1.0
cooling_rate = 0.9
max_iterations = 10
solution, cost = simulated_annealing(df_reservations, df_cottages, initial_temperature, cooling_rate, max_iterations)


In [19]:
print("Solution:", solution)
print("Cost:", cost)

Solution: [211, 318, 118, 297, 113, 701, 683, 643, 811, 187, 382, 755, 123, 519, 434, 426, 430, 442, 98, 811, 461, 451, 320, 683, 206, 303, 708, 807, 56, 18, 363, 340, 433, 766, 809, 648, 138, 118, 172, 416, 96, 215, 138, 475, 184, 224, 20, 332, 803, 722, 54, 143, 54, 640, 520, 296, 331, 358, 516, 541, 357, 112, 186, 213, 353, 135, 584, 508, 664, 466, 39, 246, 173, 545, 563, 248, 14, 278, 119, 55, 635, 483, 101, 191, 235, 8, 539, 168, 118, 32, 540, 668, 450, 644, 806, 137, 358, 547, 189, 706, 313, 104, 5, 565, 558, 117, 185, 256, 256, 711, 533, 297, 704, 699, 0, 437, 413, 337, 475, 565, 121, 751, 414, 437, 120, 399, 227, 541, 396, 206, 159, 298, 470, 678, 356, 153, 540, 475, 698, 393, 796, 537, 243, 251, 624, 64, 715, 193, 297, 41, 524, 80, 293, 798, 106, 157, 201, 246, 292, 700, 256, 160, 342, 96, 545, 466, 522, 210, 287, 462, 326, 495, 583, 530, 342, 66, 171, 653, 129, 255, 252, 339, 494, 112, 332, 435, 531, 424, 637, 465, 81, 807, 343, 325, 443, 718, 1, 708, 557, 29, 784, 256, 511, 

In [20]:

df_solution = pd.DataFrame(solution, columns=['CottageID'])

# Opslaan naar csv
df_solution.to_csv('solution.csv', index=False)